##### Deepiction V.0.4

## **Train and Test Deep Neural Networks**
- Tensorflow: unet, resnet
- Pytorch: unet_ten, unet_n2n

**1. Import the dependencies**

In [ ]:
import numpy as np
import os
from deepiction.imagedataset import ImageDataset
from deepiction.prediction import Prediction
from deepiction.training import Training
from deepiction.tools import report_resources
import warnings
warnings.filterwarnings("ignore")
report_resources()

**2. Selection of the dataset**
- task
    - binary: pixel classification, input 1 grayscale channel, output mask class=0 or class=1
    - nclass: pixel classification in n class, input 1 grayscale channel, output labels
    - regress: regression, input 1 grayscale channel, output 1 grayscale channel
- norm: (training, test)
    - minmax
    - div255
    - none

In [ ]:
#dataname, task, norm = 'simulation-object-256-8-2', 'nclass', ('minmax', 'none')
#dataname, task, norm = 'simulation-dmap-256-8-1', 'regres', ('minmax', 'none')
#dataname, task, norm = 'simulation-class-256-8-3', 'nclass', ('minmax', 'none')
#dataname, task, norm = 'degradation-noisy-p50', 'regres', ('none', 'none')
dataname, task, norm = 'simulation-binary-256-8-1', 'binary', ('minmax', 'none')
#dataname, task, norm = 'ctc-glioblastoma', 'binary', ('minmax', 'none')

datapath  = f'../datasets/{dataname}/'

# Training dataset
dataset = ImageDataset(datapath)
dataset.load_pairs(100, 'train/sources', 'train/targets')
dataset.normalization(norm)
dataset.report()

**3. Selection of the framework and the network**
- PT (tensorflow) PT-unet-ten, PT-unet-n2n
- TF (tensorflow) TF-unet, TF-resnet

In [ ]:
#netname, extension = 'PT-unet-ten', '.pt'   # not for task nclass
netname, extension = 'PT-unet-n2n', '.pt'    # not for task nclass
#netname, extension = 'TF-resnet', '.hdf5'
#netname, extension = 'TF-unet', '.hdf5'

**4. Training**

In [ ]:
epochs = 21
npools, nchannels = 3, 16
batchnorm, dropout = False, 0
batchsize, learningrate = 16, 0.001

trainname = f'{netname}-{npools}P-{nchannels}C-{epochs}E'
reportpath = f'../reports/{dataname}/{trainname}/'
pretrainedpath = f'../reports/{dataname}/{trainname}/'

if task == 'binary': activation, loss, metric, measure, noutputs = 'sigmoid', 'bce', 'mse', 'iou', 1
if task == 'nclass': activation, loss, metric, measure, noutputs = 'softmax', 'categorical_crossentropy', 'accuracy', 'iou', dataset.getNumberOfClasses()  
if task == 'regres': activation, loss, metric, measure, noutputs = 'relu', 'mse', 'mse', 'psnr', 1

dataset.split(0.25)
if task == 'nclass': dataset.makeClassesAsChannels()   
training = Training(dataset, reportpath)
training.buildnet(netname, noutputs, nchannels, npools, batchnorm, dropout, activation)

if True: # Finetuning
    training.load_pretrained(os.path.join(pretrainedpath, 'model_best' + extension))
training.train(epochs, batchsize, learningrate, loss, metric)

**5. Test**

In [ ]:
datatest = ImageDataset(datapath)   
datatest.load_pairs(5, 'test/sources', 'test/targets')
datatest.normalization(norm)
#datatest.report()

if task == 'binary': measure = 'iou'
if task == 'nclass': measure = 'iou'  
if task == 'regres': measure = 'psnr'

prediction = Prediction(datatest, reportpath, 'model_best' + extension)
preds = prediction.test()
if task == 'binary': preds = np.where(preds > 0.5, 1, 0)
if task == 'nclass': preds = np.expand_dims(np.argmax(preds, axis=3), axis=3)
prediction.print_results(preds, task != 'regres')
prediction.figure_results(preds, measure)
prediction.save_results(preds)